# "DRAFT: COVID-19 en Chile: Estadísticas demográficas"
> Total de casos confirmados, fallecidos confirmados, pacientes en UCI por sexo y rango de edad.

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/diagram.png

In [1]:
#hide
import numpy as np
import pandas as pd
import altair as alt

# Casos confirmados por grupo de edad

In [2]:
#hide
cases_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/' \
    + 'master/output/producto16/CasosGeneroEtario.csv'
)

In [3]:
#hide
today = cases_raw.iloc[:,-1].name
update_date = pd.to_datetime(today)
today

'2020-05-01'

In [6]:
#hide
cases_raw = cases_raw[['Grupo de edad', 'Sexo', today]].copy()

In [7]:
#hide
cases_raw.groupby('Sexo').sum()

,2020-05-01
Sexo,
F,8338
M,8670


In [8]:
#hide
cases_raw = cases_raw.groupby('Grupo de edad').sum()

In [9]:
#hide
cases = pd.DataFrame()
cases['Grupo de edad'] = ['<=39', '40-49', '50-59', '60-69', '70-79', '>=80']

In [10]:
#hide
cases["Casos confirmados"] = [
    cases_raw.iloc[:8].values.sum(),
    cases_raw.iloc[8:10].values.sum(),
    cases_raw.iloc[10:12].values.sum(),
    cases_raw.iloc[12:14].values.sum(),
    cases_raw.iloc[14:16].values.sum(),
    cases_raw.iloc[16:].values.sum()
]

In [11]:
#hide
total_cases = cases["Casos confirmados"].sum()
total_cases

17008

In [12]:
#hide
cases["Porcentaje"] = np.round(100*cases["Casos confirmados"]/total_cases, decimals=1)

In [13]:
#hide
cases["Texto"] = \
[f"{cases['Casos confirmados'].iloc[i]} ({cases['Porcentaje'].iloc[i]}%)" for i, val in cases.iterrows()]

In [14]:
#hide_input
bars = alt.Chart(cases.reset_index()).mark_bar(opacity=0.8, size=30).encode(
    x = alt.X('Casos confirmados'),
    y = alt.Y('Grupo de edad', sort=['<=39', '40-49', '50-59', '60-69', '70-79', '>=80']),
    tooltip = ['Grupo de edad', 'Casos confirmados', 'Porcentaje'],
    color = alt.Color('Grupo de edad', scale=alt.Scale(scheme='category10'), legend=None),
    order = alt.Order('index:O')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto')
)

bars.properties(
    title = f"Covid-19 en Chile: Casos confirmados al {pd.to_datetime(today).strftime('%d/%m/%Y')}",
    width = 600,
    height = alt.Step(40)
) + text

alt.LayerChart(...)

# Fallecidos confirmados por grupo de edad

In [15]:
#hide
deaths_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/' + \
    'master/output/producto10/FallecidosEtario.csv'
)

In [17]:
#hide
grupo_de_edad = ['<=39', '40-49', '50-59', '60-69', '70-79', '80-89', '>=90']

In [18]:
#hide
deaths = deaths_raw[['Grupo de edad', today]].copy()

In [19]:
#hide
total_deaths = deaths[today].sum()
total_deaths

234

In [20]:
#hide
deaths = deaths.rename(columns={today: "Fallecidos confirmados"})

In [21]:
#hide
deaths['Porcentaje'] = np.round(100*deaths["Fallecidos confirmados"]/total_deaths, decimals=1)

In [22]:
#hide
deaths['Texto'] = \
[f"{deaths['Fallecidos confirmados'].iloc[i]} ({deaths['Porcentaje'].iloc[i]}%)" for i, val in deaths.iterrows()]

In [23]:
#hide_input
bars = alt.Chart(deaths.reset_index()).mark_bar(opacity=0.8, size=30).encode(
    x = alt.X('Fallecidos confirmados'),
    y = alt.Y('Grupo de edad', sort=grupo_de_edad),
    tooltip = ['Grupo de edad', 'Fallecidos confirmados', 'Porcentaje'],
    color = alt.Color('Grupo de edad', scale=alt.Scale(scheme='category10'), legend=None),
    order = alt.Order('index:O')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto')
)

bars.properties(
    title = f"Covid-19 en Chile: Fallecidos confirmados al {pd.to_datetime(today).strftime('%d/%m/%Y')}",
    width = 600,
    height = alt.Step(40)
) + text

alt.LayerChart(...)

# Población por grupo de edad

In [26]:
#hide
censo_raw = pd.read_csv(
    'https://raw.githubusercontent.com/alonsosilvaallende/COVID-19/master/data/Censo_2017_Chile.csv',
    thousands='.'
)

In [27]:
#hide
censo = pd.DataFrame()
censo['Grupo de edad'] = censo_raw['Grupo de edad']

In [28]:
#hide
censo['Poblacion'] = censo_raw.drop(columns='Grupo de edad').sum(axis=1)

In [29]:
#hide
new_censo = pd.DataFrame()
new_censo['Grupo de edad'] = grupo_de_edad

In [30]:
#hide
new_censo["Poblacion"] = \
[censo.iloc[:8]["Poblacion"].sum(), \
censo.iloc[8:10]["Poblacion"].sum(), \
censo.iloc[10:12]["Poblacion"].sum(), \
censo.iloc[12:14]["Poblacion"].sum(), \
censo.iloc[14:16]["Poblacion"].sum(), \
censo.iloc[16:18]["Poblacion"].sum(), \
censo.iloc[18:]["Poblacion"].sum()]

In [31]:
#hide
total_poblacion = new_censo["Poblacion"].sum()
total_poblacion

17574003

In [32]:
#hide
new_censo["Porcentaje"] = np.round(100*new_censo["Poblacion"]/total_poblacion, decimals=1)

In [33]:
#hide
new_censo['Texto'] = \
[f"{format(new_censo['Poblacion'].iloc[i],',').replace(',', '.')} ({new_censo['Porcentaje'].iloc[i]}%)" for i, val in new_censo.iterrows()]

In [34]:
#hide_input
bars = alt.Chart(new_censo.reset_index()).mark_bar(opacity=0.8, size=30).encode(
    x = alt.X('Poblacion', axis=alt.Axis(title="Población")),
    y = alt.Y('Grupo de edad', sort=grupo_de_edad),
    tooltip = ['Grupo de edad', 'Poblacion', 'Porcentaje'],
    color = alt.Color('Grupo de edad', scale=alt.Scale(scheme='category10'), legend=None),
    order = alt.Order('index:O')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto')
)

bars.properties(
    title = f"Población Censo 2017",
    width = 600,
    height = alt.Step(40)
) + text

alt.LayerChart(...)

# Pacientes UCI por rango de edad

In [35]:
#hide
pacientes_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/'
    + 'master/output/producto9/HospitalizadosUCIEtario.csv'
)

In [36]:
#hide
pacientes = pacientes_raw[['Grupo de edad', today]]

In [37]:
#hide
total_pacientes = pacientes[today].sum()
total_pacientes

428

In [38]:
#hide
pacientes = pacientes.rename(columns={today: "Pacientes UCI"})

In [39]:
#hide
pacientes['Porcentaje'] = np.round(100*pacientes["Pacientes UCI"]/total_pacientes, decimals=1)

In [40]:
#hide
pacientes['Texto'] = \
[f"{pacientes['Pacientes UCI'].iloc[i]} ({pacientes['Porcentaje'].iloc[i]}%)" for i, val in pacientes.iterrows()]

In [41]:
#hide_input
bars = alt.Chart(pacientes.reset_index()).mark_bar(opacity=0.8, size=30).encode(
    x = alt.X('Pacientes UCI'),
    y = alt.Y('Grupo de edad', sort=grupo_de_edad),
    order = alt.Order('index:O'),
    tooltip = ['Grupo de edad', 'Pacientes UCI', 'Porcentaje'],
    color = alt.Color('Grupo de edad', scale=alt.Scale(scheme='category10'), legend=None)
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto')
)

bars.properties(
    title = f"Pacientes UCI presentes el {pd.to_datetime(today).strftime('%d/%m/%Y')}",
    width = 600,
    height = alt.Step(40)
) + text

alt.LayerChart(...)

In [42]:
#hide
# Evolución de fallecidos confirmados por rango de edad

In [43]:
#hide
data = deaths_raw.reset_index().melt(['index', 'Grupo de edad'])

In [44]:
#hide
data = data.rename(columns={'variable': "fecha (año-mes-día)", 'value': "fallecidos confirmados"})

In [45]:
#hide
input_dropdown = alt.binding_select(options=data['Grupo de edad'].unique())
selection1 = alt.selection_single(fields=['Grupo de edad'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Grupo de edad'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Grupo de edad:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(data).mark_bar().encode(
    x = 'fecha (año-mes-día):N',
    y = 'fallecidos confirmados:Q',
    color = color,
    tooltip = ['fecha (año-mes-día)', 'Grupo de edad', 'fallecidos confirmados'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'index',
    sort='ascending'
    )
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(data).mark_point().encode(
    y=alt.Y('Grupo de edad', axis=alt.Axis(orient='right'), sort=grupo_de_edad),
    color=color,
    order=alt.Order(
    # Sort the segments of the bars by this field
    'index',
    sort='descending'
    )
).add_selection(
    selection1, selection2
)

bars.properties(
    title = 'COVID-19 en Chile: Evolución de fallecidos confirmados por rango de edad',
    width = 600
) | legend

alt.HConcatChart(...)